In [1]:
# load and prepare all of 4 folds data
import pandas as pd

def load_and_filter_fold(i):
    train_dir = f'../../cleaned_dataset/split_data/fold{i}/train.csv'  
    test_dir = f'../../cleaned_dataset/split_data/fold{i}/test.csv'   
    # load the data 
    train_df = pd.read_csv(train_dir)
    test_df = pd.read_csv(test_dir)

    # Get all of the unique labels for train / test sets 
    train_labels = list(train_df['room'].unique())
    test_labels = list(test_df['room'].unique())

    # Labels that appear in BOTH train and test
    common_labels = list(set(train_labels) & set(test_labels))

    # Filter to just keep the records with labels in common labels list
    train_df = train_df[train_df['room'].isin(common_labels)].reset_index(drop=True)
    test_df  = test_df[test_df['room'].isin(common_labels)].reset_index(drop=True)

    return train_df, test_df

# Load fold 1
train_df_1, test_df_1 = load_and_filter_fold(1)

# Load fold 2
train_df_2, test_df_2 = load_and_filter_fold(2)

# Load fold 3
train_df_3, test_df_3 = load_and_filter_fold(3)

# Load fold 4
train_df_4, test_df_4 = load_and_filter_fold(4)

In [2]:
# Function to add 25 beacon_1, beacon_2, ... , beacon_25
def add_beacon_features(df, num_beacons=25):
    df = df.copy()

    for i in range(1, num_beacons + 1):
        df[f'beacon_{i}'] = df['RSSI'].where(df['mac address'] == i, 0)

    return df

def aggregate_by_timestamp(df):
    """
    Aggregate beacon data by timestamp (1-second windows)
    
    For each beacon column:
    - If ALL values are 0: set mean, std, min, max, count = 0
    - If ANY non-zero values exist: calculate statistics ONLY on non-zero values
    
    Args:
        df: DataFrame with columns: timestamp, room, beacon_1, beacon_2, ..., beacon_25
    
    Returns:
        windowed_df: DataFrame with columns:
            - timestamp
            - room
            - beacon_1_mean, beacon_1_std, beacon_1_min, beacon_1_max, beacon_1_count
            - beacon_2_mean, beacon_2_std, beacon_2_min, beacon_2_max, beacon_2_count
            - ...
            - beacon_25_mean, beacon_25_std, beacon_25_min, beacon_25_max, beacon_25_count
    """    
    df = df.copy()
    
    # Group by timestamp
    grouped = df.groupby('timestamp')
    
    result_rows = []
    
    for timestamp, group in grouped:
        # Initialize row with timestamp and room
        row_data = {
            'timestamp': timestamp,
            'room': group['room'].iloc[0]  # Take first room value (should be consistent)
        }
        
        # Process each beacon column (beacon_1 to beacon_25)
        for beacon_id in range(1, 26):
            beacon_col = f'beacon_{beacon_id}'
            
            # Get all values for this beacon in this window
            beacon_values = group[beacon_col].values
            
            # Filter to get only non-zero values
            non_zero_values = beacon_values[beacon_values != 0]
            
            # Check if we have any non-zero values
            if len(non_zero_values) > 0:
                # Calculate statistics on non-zero values ONLY
                row_data[f'{beacon_col}_mean'] = non_zero_values.mean()
                row_data[f'{beacon_col}_std'] = non_zero_values.std() if len(non_zero_values) > 1 else 0.0
                row_data[f'{beacon_col}_min'] = non_zero_values.min()
                row_data[f'{beacon_col}_max'] = non_zero_values.max()
                row_data[f'{beacon_col}_count'] = len(non_zero_values)
            else:
                # All values are zero - set everything to 0
                row_data[f'{beacon_col}_mean'] = 0.0
                row_data[f'{beacon_col}_std'] = 0.0
                row_data[f'{beacon_col}_min'] = 0.0
                row_data[f'{beacon_col}_max'] = 0.0
                row_data[f'{beacon_col}_count'] = 0
        
        result_rows.append(row_data)
    
    # Create DataFrame from result rows
    windowed_df = pd.DataFrame(result_rows)
    
    # Filter out completely empty windows (all beacons are 0)
    beacon_mean_cols = [f'beacon_{i}_mean' for i in range(1, 26)]
    valid_windows = windowed_df[beacon_mean_cols].sum(axis=1) != 0
    
    removed_count = (~valid_windows).sum()
    windowed_df = windowed_df[valid_windows].reset_index(drop=True)
    
    print(f"Total windows after aggregation: {len(windowed_df)}")
    print(f"Removed {removed_count} empty windows (all beacons = 0)")
    print(f"Features: 25 beacons × 5 statistics = 125 features")
    
    return windowed_df

In [3]:
# import essential libraries for model 
import xgboost as xgb
from sklearn.metrics import classification_report, f1_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_sample_weight
import warnings
warnings.filterwarnings('ignore')

In [4]:
# Install and import SMOTE
from imblearn.over_sampling import SMOTE
from collections import Counter

def analyze_and_apply_smote(X_train, y_train_encoded, label_encoder, k_neighbors=5):
    """
    Analyze class distribution, identify 3 smallest minority classes, 
    and apply SMOTE to balance them.
    
    Args:
        X_train: Training features
        y_train_encoded: Encoded training labels
        label_encoder: LabelEncoder object
        k_neighbors: Number of neighbors for SMOTE (default=5)
    
    Returns:
        X_resampled, y_resampled: Balanced training data
    """
    # Count classes before SMOTE
    class_counts_before = Counter(y_train_encoded)
    
    print("\n" + "="*60)
    print("CLASS DISTRIBUTION ANALYSIS")
    print("="*60)
    
    # Sort classes by count (ascending)
    sorted_classes = sorted(class_counts_before.items(), key=lambda x: x[1])
    
    print("\nAll classes (sorted by count):")
    for class_idx, count in sorted_classes:
        class_name = label_encoder.inverse_transform([class_idx])[0]
        print(f"  {class_name:20s}: {count:4d} samples")
    
    # Identify 3 smallest minority classes
    three_smallest = sorted_classes[:3]
    print("\n🎯 3 SMALLEST MINORITY CLASSES:")
    for class_idx, count in three_smallest:
        class_name = label_encoder.inverse_transform([class_idx])[0]
        print(f"  {class_name:20s}: {count:4d} samples")
    
    # Check if SMOTE is applicable
    min_class_count = sorted_classes[0][1]
    if min_class_count < k_neighbors + 1:
        k_neighbors = max(1, min_class_count - 1)
        print(f"\n⚠️  Adjusted k_neighbors to {k_neighbors} (smallest class has {min_class_count} samples)")
    
    # Apply SMOTE
    print(f"\nApplying SMOTE with k_neighbors={k_neighbors}...")
    smote = SMOTE(random_state=42, k_neighbors=k_neighbors)
    X_resampled, y_resampled = smote.fit_resample(X_train, y_train_encoded)
    
    # Count classes after SMOTE
    class_counts_after = Counter(y_resampled)
    
    print("\n" + "="*60)
    print("AFTER SMOTE")
    print("="*60)
    sorted_classes_after = sorted(class_counts_after.items(), key=lambda x: x[1])
    for class_idx, count in sorted_classes_after:
        class_name = label_encoder.inverse_transform([class_idx])[0]
        before_count = class_counts_before[class_idx]
        change = count - before_count
        print(f"  {class_name:20s}: {before_count:4d} → {count:4d} (+{change:4d})")
    
    print(f"\nTotal samples: {len(y_train_encoded)} → {len(y_resampled)}")
    print("="*60 + "\n")
    
    return X_resampled, y_resampled

In [5]:
def train_evaluate_fold(i, train_df, test_df):
    # firstly get the train and test df
    print(f"Loading the data frame from fold {i}")
    
    # secondly create 25 beacon vector features 
    print("Adding 25 beacon vector features for both set")
    train_df = add_beacon_features(train_df)
    test_df = add_beacon_features(test_df)

    # apply windowing to both of the set 
    print("Applying windowing for both sets")
    windowed_train_df = aggregate_by_timestamp(train_df)
    windowed_test_df = aggregate_by_timestamp(test_df)

    # prepare training data
    feature_cols = [col for col in windowed_train_df.columns 
                    if col not in ['room', 'timestamp']]

    X_train = windowed_train_df[feature_cols]
    y_train = windowed_train_df['room']

    # Encode labels for XGBoost (XGBoost requires numeric labels)
    label_encoder = LabelEncoder()
    y_train_encoded = label_encoder.fit_transform(y_train)

    # ===== NEW: APPLY SMOTE HERE =====
    X_train_resampled, y_train_resampled = analyze_and_apply_smote(
        X_train, 
        y_train_encoded, 
        label_encoder,
        k_neighbors=5
    )
    
    # Calculate sample weights on resampled data
    sample_weights = compute_sample_weight(class_weight='balanced', y=y_train_resampled)

    # Train XGBoost with resampled data
    xgb_model = xgb.XGBClassifier(
        n_estimators=100,
        max_depth=10,
        learning_rate=0.1,
        min_child_weight=1,
        subsample=0.8,
        colsample_bytree=0.8,
        gamma=0,
        reg_alpha=0,
        reg_lambda=1,
        objective='multi:softmax',
        num_class=len(label_encoder.classes_),
        eval_metric='mlogloss',
        random_state=42,
        n_jobs=-1,
        tree_method='hist'
    )

    print("Training XGBoost with SMOTE-balanced data...")
    xgb_model.fit(
        X_train_resampled,  # Use resampled data
        y_train_resampled,  # Use resampled labels
        sample_weight=sample_weights
    )
    print("Training completed!")
    
    # Prepare test data (no changes here)
    X_test = windowed_test_df[feature_cols]
    y_test = windowed_test_df['room']
    y_test_encoded = label_encoder.transform(y_test)
    
    # Make predictions
    print("Making predictions...")
    y_pred_encoded = xgb_model.predict(X_test)
    y_pred = label_encoder.inverse_transform(y_pred_encoded)
    
    # Calculate macro F1 score
    macro_f1 = f1_score(y_test, y_pred, average='macro')
    
    # Calculate per-class F1 scores
    per_class_f1 = f1_score(y_test, y_pred, average=None, labels=label_encoder.classes_)
    per_class_f1_dict = {label: f1 for label, f1 in zip(label_encoder.classes_, per_class_f1)}
    
    print(f"Fold {i} - Macro F1 Score: {macro_f1:.4f}")
    
    return {
        'fold': i,
        'macro_f1': macro_f1,
        'per_class_f1': per_class_f1_dict,
        'classes': label_encoder.classes_
    }

In [6]:
# Train and evaluate all folds
results = {}

folds = {
    1: (train_df_1, test_df_1),
    2: (train_df_2, test_df_2),
    3: (train_df_3, test_df_3),
    4: (train_df_4, test_df_4)
}

for fold_num, (train_df, test_df) in folds.items():
    print(f"\n{'='*80}")
    print(f"PROCESSING FOLD {fold_num}")
    print(f"{'='*80}\n")
    
    result = train_evaluate_fold(fold_num, train_df, test_df)
    results[fold_num] = result
    
print("\n" + "="*80)
print("ALL FOLDS COMPLETED!")
print("="*80)


PROCESSING FOLD 1

Loading the data frame from fold 1
Adding 25 beacon vector features for both set
Applying windowing for both sets
Total windows after aggregation: 19280
Removed 0 empty windows (all beacons = 0)
Features: 25 beacons × 5 statistics = 125 features
Total windows after aggregation: 2481
Removed 0 empty windows (all beacons = 0)
Features: 25 beacons × 5 statistics = 125 features

CLASS DISTRIBUTION ANALYSIS

All classes (sorted by count):
  505                 :    4 samples
  508                 :   17 samples
  517                 :   17 samples
  518                 :   18 samples
  515                 :   39 samples
  506                 :  162 samples
  513                 :  195 samples
  520                 :  260 samples
  523                 :  342 samples
  cleaning            :  583 samples
  cafeteria           : 4568 samples
  kitchen             : 4588 samples
  nurse station       : 8487 samples

🎯 3 SMALLEST MINORITY CLASSES:
  505                 :    4 

In [7]:
import numpy as np

# Save results to text file
with open('startpipeline_smote.txt', 'w') as f:
    f.write("="*80 + "\n")
    f.write("CROSS-VALIDATION RESULTS - MACRO F1 SCORES\n")
    f.write("="*80 + "\n\n")
    
    # Overall macro F1 scores
    f.write("MACRO F1 SCORES PER FOLD:\n")
    f.write("-"*80 + "\n")
    for fold_num in [1, 2, 3, 4]:
        f.write(f"Fold {fold_num}: {results[fold_num]['macro_f1']:.4f}\n")
    
    # Calculate mean and std
    macro_f1_scores = [results[i]['macro_f1'] for i in [1, 2, 3, 4]]
    mean_f1 = np.mean(macro_f1_scores)
    std_f1 = np.std(macro_f1_scores)
    
    f.write(f"\nMean Macro F1: {mean_f1:.4f} ± {std_f1:.4f}\n")
    f.write(f"Min: {np.min(macro_f1_scores):.4f}, Max: {np.max(macro_f1_scores):.4f}\n")
    
    # Per-class F1 scores for each fold
    f.write("\n" + "="*80 + "\n")
    f.write("PER-CLASS F1 SCORES\n")
    f.write("="*80 + "\n\n")
    
    for fold_num in [1, 2, 3, 4]:
        f.write(f"\nFold {fold_num}:\n")
        f.write("-"*80 + "\n")
        per_class = results[fold_num]['per_class_f1']
        for class_name in sorted(per_class.keys()):
            f.write(f"{class_name:20s}: {per_class[class_name]:.4f}\n")

print("✅ Results saved to startpipeline_smote.txt")

✅ Results saved to startpipeline_smote.txt


In [8]:
# Display summary
print("\n" + "="*80)
print("SUMMARY")
print("="*80)

for fold_num in [1, 2, 3, 4]:
    print(f"\nFold {fold_num}: Macro F1 = {results[fold_num]['macro_f1']:.4f}")

macro_f1_scores = [results[i]['macro_f1'] for i in [1, 2, 3, 4]]
print(f"\n{'='*80}")
print(f"Mean Macro F1: {np.mean(macro_f1_scores):.4f} ± {np.std(macro_f1_scores):.4f}")
print(f"{'='*80}")


SUMMARY

Fold 1: Macro F1 = 0.2945

Fold 2: Macro F1 = 0.2682

Fold 3: Macro F1 = 0.2709

Fold 4: Macro F1 = 0.3090

Mean Macro F1: 0.2857 ± 0.0169
